### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [ ]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [1]:
from datetime import datetime as t

def build_prefix_table(pattern): # Построение префикс таблицы
    prefix_table = [0] * len(pattern)
    j = 0
    for i in range(1, len(pattern)):
        while j > 0 and pattern[i] != pattern[j]:
            j = prefix_table[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        prefix_table[i] = j
    return prefix_table

def kmp_search(text, pattern):
    prefix_table = build_prefix_table(pattern) # Построить префикс таблицу для подстроки
    n = len(text) # Длина строки
    m = len(pattern) # Длина подстроки
    if m == 0:
        return []
    q = 0  # Количество совпавших символов
    result = []
    for i in range(n):
        while q > 0 and pattern[q] != text[i]:
            q = prefix_table[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == m:
            # Найдено вхождение
            result.append(i - m + 1)
            # Продолжаем поиск следующих вхождений
            q = prefix_table[q - 1]
    
    return result

text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
sensitive = input("Учитывать регистр? (да/нет): ")
if sensitive.lower() == "нет":
    text.lower()
    pattern.lower()

def out(index,start_time):
    if index != -1:
        print(f"Подстрока найдена в позиции {index} в строке.")
    else:
        print("Подстрока не найдена в строке.")
    time = t.now() - start_time
    print(f"Время выполнения: {time}")

start_time = t.now()
index = kmp_search(text, pattern)
out(index,start_time)

Подстрока найдена в позиции [5] в строке.
Время выполнения: 0:00:00.001199


#### Упрощенный алгоритм Бойера-Мура

In [2]:
from datetime import datetime as t
from collections import defaultdict

def build_shift_table(pattern):
    m = len(pattern)
    table = defaultdict(lambda: m)
    for i in range(m - 1):
        table[pattern[i]] = m - i - 1
    return table

def boyer_moore_search(text, pattern):
    n = len(text)
    m = len(pattern)
    if m == 0:
        return []
    
    shift_table = build_shift_table(pattern)
    result = []
    i = 0

    while i <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[i + j]:
            j -= 1
        if j < 0:
            result.append(i)
            i += m
        else:
            i += shift_table[text[i + m - 1]]
    return result

text = input("Введите строку: ")
pattern = input("Введите подстроку: ")
sensitive = input("Учитывать регистр? (да/нет): ")

if sensitive.lower() == "нет":
    text = text.lower()
    pattern = pattern.lower()

start = t.now()
positions = boyer_moore_search(text, pattern)
elapsed = t.now() - start

if positions:
    print(f"Подстрока найдена в позициях: {positions}")
else:
    print("Подстрока не найдена.")
print(f"Время выполнения: {elapsed.total_seconds():.8f} сек")

Подстрока найдена в позициях: [0, 4]
Время выполнения: 0.00021900 сек


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [ ]:
from collections import deque

# Проверка на решаемость
def is_solvable(puzzle):
    inv_count = 0
    for i in range(len(puzzle)):
        for j in range(i + 1, len(puzzle)):
            if puzzle[i] != 0 and puzzle[j] != 0 and puzzle[i] > puzzle[j]:
                inv_count += 1
    row_from_bottom = 3 - (puzzle.index(0) // 4)
    return (inv_count + row_from_bottom) % 2 == 0

# Получаем все возможные ходы
def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, 4)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # вверх, вниз, влево, вправо

    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 4 and 0 <= new_col < 4:
            new_index = new_row * 4 + new_col
            new_state = list(state)
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append((tuple(new_state), new_state[zero_index]))  # Состояние и фишка, которую двигали
    return neighbors

# Решение через BFS
def solve(puzzle):
    if not is_solvable(puzzle):
        return []

    start = tuple(puzzle)
    goal = tuple(range(1, 16)) + (0,)
    queue = deque([(start, [])])  # Очередь для BFS
    visited = set()

    while queue:
        current, path = queue.popleft()

        if current == goal:
            return path  # Возвращаем путь, как только нашли решение

        if current in visited:
            continue
        visited.add(current)

        for neighbor, moved_tile in get_neighbors(current):
            if neighbor not in visited:
                queue.append((neighbor, path + [moved_tile]))

    return []

# Ввод и запуск программы
puzzle_input = input("Введите 16 чисел (от 0 до 15), разделённых пробелами:\n")
puzzle = list(map(int, puzzle_input.strip().split()))

moves = solve(puzzle)

if moves:
    print("\nРешение найдено!")
    print("Последовательность движений:", moves)
else:
    print("\nРешения нет.")

### Вывод